 # Pipeline
region Data Sources

 ## Data Sources
region Beer Advocate
  --------------------------------  Beers  
region Beers
endregion
  --------------------------------  Breweries  
region Breweries
endregion
  --------------------------------  Users  
region Users
endregion
  --------------------------------  Ratings  
region Ratings
endregion
  --------------------------------  Reviews  
region Reviews
endregion
endregion
   --------------------------------   Rate Beer   --------------------------------   
region Rate Beer
endregion
   --------------------------------   Matched Dataset   --------------------------------   
region Matched Dataset
endregion
endregion
   --------------------------------   Data Model   --------------------------------   
   ------------------------------------------------------------------------------   
region Data Model

 ## Data Model
endregion
   --------------------------------   Data Wrangling   --------------------------------   
   ------------------------------------------------------------------------------   
region Data Wrangling

 ## Data Wrangling
endregion
endregion